In [2]:
import site
site.addsitedir("/rhome/kzhao028/.local/lib/python3.11/site-packages")
import pandas as pd
import numpy as np
import os
import glob
import gffutils

import seaborn as sns
import scipy as scipy
import sys
import matplotlib
matplotlib.style.use('ggplot')
idx = pd.IndexSlice

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [10]:
fqlib=glob.glob('/rhome/kzhao028/shared/seqdata/20251202*/*fastq.gz')
print(fqlib[0])
bamlist=glob.glob('./bamfiles_hg38/*bam')

/rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/D25XYN_2_mCh_rep2.fastq.gz


In [13]:
import os

samplename_list = []
sname_list = []

def strip_fastq_ext(fname):
    """Remove .fastq.gz or .fq.gz but keep entire basename."""
    if fname.endswith(".fastq.gz"):
        return fname[:-9]
    if fname.endswith(".fq.gz"):
        return fname[:-6]
    return os.path.splitext(fname)[0]

# Build sample lists
for l in fqlib:
    if 'EKDL' not in l:
        base = os.path.basename(l)
        name = strip_fastq_ext(base)
        samplename_list.append(name)
        sname_list.append(name)

# Build list of existing BAM files to skip
bname_list = []
for b in bamlist:
    if 'EKDL25' not in b:
        base = os.path.basename(b)
        name = strip_fastq_ext(base)
        bname_list.append(name)

# Identify samples that still need STAR alignment
the_rest_list = list(set(sname_list).difference(bname_list))
print("Samples to process:", len(the_rest_list))
print("Existing BAMs:", len(bname_list))

# Output directory
outdir = "/rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/star_output/"
indir  = "/rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/"

# Build STAR commands for single-end reads
cmdlist = []
for sample in the_rest_list:

    fastq_path = os.path.join(indir, sample + ".fastq.gz")
    prefix = os.path.join(outdir, sample + "_")

    starcmd = (
        'STAR --genomeDir /rhome/kzhao028/shared/Joy/references/human_flu/fluWSN_nicole_hg38v47 '
        '--readFilesIn ' + fastq_path +
        ' --readFilesCommand zcat '
        '--outFileNamePrefix ' + prefix +
        ' --outSAMtype BAM SortedByCoordinate --runThreadN 16\n'
    )
     #   'mv ' + prefix + '*out* '
      #  '/rhome/jshengna/shared/Joy/20250419-IRD_rna_editing/star_output/\n'
       # )


    print(starcmd)
    cmdlist.append(starcmd)


Samples to process: 15
Existing BAMs: 0
STAR --genomeDir /rhome/kzhao028/shared/Joy/references/human_flu/fluWSN_nicole_hg38v47 --readFilesIn /rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/D25XYN_1_mCh_rep1.fastq.gz --readFilesCommand zcat --outFileNamePrefix /rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/star_output/D25XYN_1_mCh_rep1_ --outSAMtype BAM SortedByCoordinate --runThreadN 16

STAR --genomeDir /rhome/kzhao028/shared/Joy/references/human_flu/fluWSN_nicole_hg38v47 --readFilesIn /rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/D25XYN_5_PB1_rep2.fastq.gz --readFilesCommand zcat --outFileNamePrefix /rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/star_output/D25XYN_5_PB1_rep2_ --outSAMtype BAM SortedByCoordinate --runThreadN 16

STAR --genomeDir /rhome/kzhao028/shared/Joy/references/human_flu/fluWSN_nicole_hg38v47 --readFilesIn /rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/D25XYN_2_mCh_rep2.fastq.gz --readFilesCommand zcat --outFileNamePre

In [43]:
f=open('run_the_rest_batch.sh','w')

#!/bin/bash -l

#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=10
#SBATCH --mem=10G
#SBATCH --time=1-00:15:00     # 1 day and 15 minutes
#SBATCH --mail-user=useremail@address.com
#SBATCH --mail-type=ALL
#SBATCH --job-name="just_a_test"
#SBATCH -p epyc # You can use any of the following; epyc, intel, batch, highmem, gpu


f.write('#!/bin/bash -l\n')
f.write('#SBATCH --nodes=1\n')
f.write('#SBATCH --cpus-per-task=16\n')
f.write('#SBATCH --mem=128G\n')
f.write('#SBATCH --time=2-00:00:00\n')
f.write('#SBATCH --job-name="ird"\n')
f.write('#SBATCH -p highmem\n')
f.write('module load star\n')
f.write('module load samtools\n')

for c in cmdlist:
    print(c)
    f.write(c)

f.close()

15

18

27

19

26

...

445

STAR --genomeDir /rhome/jshengna/shared/Joy/references/hg38/hg38_STAR_index --readFilesIn /rhome/jshengna/shared/seqdata/20250407-IRD_RNA_editing/IRD22_cr58_r2_1.fq.gz /rhome/jshengna/shared/seqdata/20250407-IRD_RNA_editing/IRD22_cr58_r2_2.fq.gz --readFilesCommand zcat --outFileNamePrefix IRD22_cr58_r2_ --outSAMtype BAM SortedByCoordinate --runThreadN 16
mv IRD22_cr58_r2_*out* /rhome/jshengna/shared/Joy/20250419-IRD_rna_editing/star_output/



445

STAR --genomeDir /rhome/jshengna/shared/Joy/references/hg38/hg38_STAR_index --readFilesIn /rhome/jshengna/shared/seqdata/20250407-IRD_RNA_editing/IRD22_cr61_r1_1.fq.gz /rhome/jshengna/shared/seqdata/20250407-IRD_RNA_editing/IRD22_cr61_r1_2.fq.gz --readFilesCommand zcat --outFileNamePrefix IRD22_cr61_r1_ --outSAMtype BAM SortedByCoordinate --runThreadN 16
mv IRD22_cr61_r1_*out* /rhome/jshengna/shared/Joy/20250419-IRD_rna_editing/star_output/



445

In [24]:
bamlist=glob.glob('./bamfiles_hg38/*bam')
len(bamlist)
updated_cmdlist=[]
for i,c in enumerate(cmdlist):
    for b in bamlist:
        bname='_'.join(os.path.basename(b).split('_')[:-1])
        # print(bname)
        if bname not in c:
            updated_cmdlist.append(c)

19

In [ ]:
len(cmdlist)
updated_cmdlist=cmdlist

bamlist=glob.glob('./bamfiles_hg38/*bam')
len(bamlist)
# print(bamlist)

for b in bamlist:
    bname='_'.join(os.path.basename(b).split('_')[:-1])
    # print(bname)
    for c in cmdlist:
        if bname not in c:
            updated_cmdlist.append(c)
f.close()

print(updated_cmdlist)

72

19

In [20]:
f=open('run_the_rest.sh','w')
f.write('module load star\n')
f.write('module load samtools\n')
for c in updated_cmdlist:
    print(c)
    f.write(c)

f.close()


17

21

STAR --genomeDir /rhome/jshengna/shared/Joy/references/hg38/hg38_STAR_index --readFilesIn mut1804_cr61_r2_1.fq.gz mut1804_cr61_r2_2.fq.gz --readFilesCommand zcat --outFileNamePrefix mut1804_cr61_r2_ --outSAMtype BAM SortedByCoordinate --runThreadN 6



250

STAR --genomeDir /rhome/jshengna/shared/Joy/references/hg38/hg38_STAR_index --readFilesIn mut1804_cr61_r2_1.fq.gz mut1804_cr61_r2_2.fq.gz --readFilesCommand zcat --outFileNamePrefix mut1804_cr61_r2_ --outSAMtype BAM SortedByCoordinate --runThreadN 6



...

250

STAR --genomeDir /rhome/jshengna/shared/Joy/references/hg38/hg38_STAR_index --readFilesIn mut1804_cr58_r2_1.fq.gz mut1804_cr58_r2_2.fq.gz --readFilesCommand zcat --outFileNamePrefix mut1804_cr58_r2_ --outSAMtype BAM SortedByCoordinate --runThreadN 6



250

STAR --genomeDir /rhome/jshengna/shared/Joy/references/hg38/hg38_STAR_index --readFilesIn mut1804_cr58_r2_1.fq.gz mut1804_cr58_r2_2.fq.gz --readFilesCommand zcat --outFileNamePrefix mut1804_cr58_r2_ --outSAMtype BAM SortedByCoordinate --runThreadN 6



250